In [ ]:
using LightGraphs
using LinearAlgebra
using SparseArrays
using Statistics
using NLsolve
using DelimitedFiles

using Calculus

In [ ]:
g = SimpleGraph(5)
add_edge!(g, 1, 2)
add_edge!(g, 1, 3)
add_edge!(g, 2, 3)
add_edge!(g, 2, 4)
add_edge!(g, 3, 5)

E = incidence_matrix(g, oriented=true)

sources = [-4.0, 1, 1, 1, 1]
Array(E)

In [ ]:
function P(k, κ)
    P = calculate_mean_P(k, κ)
    P
end

function grad_P(k, κ)
    mean_Δp_sqr = calculate_mean_Δp_sqr(k, κ)
    
    -mean_Δp_sqr
end

function hess_P(k, κ)
    Δp = calculate_mean_Δp(k, κ)
    
    2Δp
end

In [ ]:
function calculate_mean_flows(k, κ)
    flows = zeros(ne(g))
    
    for e=1:ne(g)
        k_dmg = copy(k)
        k_dmg[e] *= κ

        K_mat = spdiagm(0 => k_dmg)
        # compute flows on graph
        L = Array(E*K_mat*E')
        
        p = L[2:end,2:end] \ sources[2:end]

        flows .+= (K_mat*E'[:,2:end]*p).^2
    end

    flows/ne(g)
end

function calculate_mean_Δp_sqr(k, κ)
    Δp_sqr = zeros(ne(g))
    
    for e=1:ne(g)
        k_dmg = copy(k)
        k_dmg[e] *= κ
        
        κs = ones(length(k))
        κs[e] = κ
        
        K_mat = spdiagm(0 => k_dmg)
        # compute flows on graph
        L = Array(E*K_mat*E')
        
        Δp = E'[:,2:end]*(L[2:end,2:end] \ sources[2:end])
        
        Δp_sqr .+= κs.*(Δp.^2)
    end

    Δp_sqr/ne(g)
end

function calculate_mean_P(k, κ)
    P = 0.0
    
    for e=1:ne(g)
        k_dmg = copy(k)
        k_dmg[e] *= κ

        K_mat = spdiagm(0 => k_dmg)
        # compute flows on graph
        L = Array(E*K_mat*E')
        
        Δp = E'[:,2:end]*(L[2:end,2:end] \ sources[2:end])
        
        P += k_dmg'*(Δp.^2)
    end

    P/ne(g)
end

function calculate_mean_Δp(k, κ)
    flows = zeros(ne(g))
    Δp = zeros(ne(g), ne(g))
    
    for e=1:ne(g)
        k_dmg = copy(k)
        k_dmg[e] *= κ
        
        κs = ones(length(k))
        κs[e] = κ

        K_mat = spdiagm(0 => k_dmg)
        # compute flows on graph
        L = Array(E*K_mat*E')
        
        Q = E'[:,2:end]*(L[2:end,2:end] \ Array(E[2:end,:]))
        dp = κs.*(E'[:,2:end]*(L[2:end,2:end] \ sources[2:end]))
        
        Δp .+= (dp*dp').*Q
    end
    
    Δp/ne(g)
end

function lagrangian_grad_tree!(F, x; κ=0.2, γ=0.5, P_c=1, return_P=false)
    """ The gradient of the Lagrangian for the damage optimization functional
    """
    k = abs.(x[1:end-1])
    λ = x[end]
    
    k = [k[1], k[2], 0.0, k[4], k[5]]
        
    gp = grad_P(k, κ)
    
    F[1:end-1] .= -(k).*gp + λ*γ*k.^(γ)
    F[3] = x[3]
    F[end] = P(k, κ) - P_c
    
    if return_P
        return sum(k.^γ), P(k, κ)
    end
end

function lagrangian_grad!(F, x; κ=0.2, γ=0.5, P_c=1, return_P=false)
    """ The gradient of the Lagrangian for the damage optimization functional
    """
    k = abs.(x[1:end-1])
    λ = x[end]
    
    gp = grad_P(k, κ)
    
    F[1:end-1] .= -λ*(k).*gp + γ*k.^(γ)
    F[end] = P(k, κ) - P_c
    
    if return_P
        return sum(k.^γ), P(k, κ)
    end
end

# try random initial conditions
function sample_critical_points(κ, γ; N=100)
    # symmetric solutions only
    lg!(F, x; return_P=false) = lagrangian_grad!(F, x; κ=κ, γ=γ, return_P=return_P)
    lg_t!(F, x; return_P=false) = lagrangian_grad_tree!(F, x; κ=κ, γ=γ, return_P=return_P)
    
    slns = []
    Ps = []
    for i_zero=1:3
        for i=1:N
            # non-tree
            x0 = 2randn(ne(g) + 1)
            x0[i_zero] = 0.0
            sln = nlsolve(lg!, x0; iterations=1000, ftol=1e-14, xtol=0.0)
            
            if converged(sln) && solution_is_symmetric(sln)
                push!(slns, sln)

                F = similar(x0)
                K, P = lg!(F, sln.zero; return_P=true)
                push!(Ps, K)
            end
            
            # tree
            x0 = 2randn(ne(g) + 1)
            sln = nlsolve(lg_t!, x0; iterations=1000, ftol=1e-14, xtol=0.0)
            
            if converged(sln) && solution_is_symmetric(sln)
                push!(slns, sln)

                F = similar(x0)
                K, P = lg!(F, sln.zero; return_P=true)
                push!(Ps, K)
            end            
        end
    end
    
    v = round.(Ps, digits=4)
    uinds = indexin(unique(v), v)
    @show unique(v)
    
    Ps[uinds], slns[uinds]
end

In [ ]:
κs = LinRange(0.02, 0.2, 220)
γs = LinRange(0.4, 0.7, 220)

In [ ]:
γ = 0.5

Pps = []
slns = []
# for κ in κs
for κ in κs
    @show κ
    Ps, sln = sample_critical_points(κ, γ; N=30)
    push!(Pps, Ps)
    push!(slns, sln)
end

In [ ]:
κ = 0.15

Pps_kappa = []
slns_kappa = []
# for κ in κs
for γ in γs
    @show γ
    Ps, class, sln = sample_critical_points(κ, γ; N=30)
    push!(Pps_kappa, Ps)
    push!(slns_kappa, sln)
end

In [ ]:
# separate the different networks
function separate(Pps, slns)
    P_tree = []
    sln_tree = []

    P_loop_1 = []
    sln_loop_1 = []

    P_loop_2 = []
    sln_loop_2 = []

    for (Ps, sln) in zip(Pps, slns)

        P_loop = []
        sln_loop = []
        found_tree = false
        for (P, s) in zip(Ps, sln)
#             @show s.zero
            if (s.zero[3] < 1e-4) && !found_tree
                # this is a symmetric tree network
                push!(P_tree, P)
                push!(sln_tree, s)
                found_tree = true
            else
                # this is a loopy network
                push!(P_loop, P)
                push!(sln_loop, s)            
            end
        end
        
        if !found_tree
            push!(P_tree, -1)
            push!(sln_tree, nothing)
        end
        
        if length(P_loop) >= 2
            idx = sortperm(P_loop)
            @show P_loop

            push!(P_loop_1, P_loop[idx[1]])
            push!(sln_loop_1, sln_loop[idx[1]])

            push!(P_loop_2, P_loop[idx[2]])
            push!(sln_loop_2, sln_loop[idx[2]])
        else
            push!(P_loop_1, NaN)
            push!(sln_loop_1, nothing)

            push!(P_loop_2, NaN)
            push!(sln_loop_2, nothing)        
        end
    end
   
    P_tree, P_loop_1, P_loop_2,  sln_tree, sln_loop_1, sln_loop_2
end

P_tree, P_loop_1, P_loop_2,  sln_tree, sln_loop_1, sln_loop_2 = separate(Pps, slns)
P_tree_k, P_loop_1_k, P_loop_2_k,  sln_tree_k, sln_loop_1_k, sln_loop_2_k = separate(Pps_kappa, slns_kappa);

In [ ]:
# save data

kappa_data = zeros(length(γs), 19)
kappa_data[:,1] = collect(γs)#
kappa_data[:,2] = P_tree_k
kappa_data[:,3] = P_loop_1_k
kappa_data[:,4] = P_loop_2_k
kappa_data[:,5:9]   = vcat([(sln != nothing) ? abs.(sln.zero)[1:5]' : [NaN, NaN, NaN, NaN, NaN]' for sln in sln_tree_k]...)
kappa_data[:,10:14]  = vcat([(sln != nothing) ? abs.(sln.zero)[1:5]' : [NaN, NaN, NaN, NaN, NaN]' for sln in sln_loop_1_k]...)
kappa_data[:,15:19]  = vcat([(sln != nothing) ? abs.(sln.zero)[1:5]' : [NaN, NaN, NaN, NaN, NaN]' for sln in sln_loop_2_k]...)

open("Delta_$(1-κ)_gamma_varies_fixed_P.txt", "w") do io
           writedlm(io, kappa_data)
end

gamma_data = zeros(length(κs), 19)
gamma_data[:,1] = 1 .- collect(κs)
gamma_data[:,2] = P_tree
gamma_data[:,3] = P_loop_1
gamma_data[:,4] = P_loop_2
gamma_data[:,5:9]   = vcat([(sln != nothing) ? abs.(sln.zero)[1:5]' : [NaN, NaN, NaN, NaN, NaN]' for sln in sln_tree]...)
gamma_data[:,10:14]  = vcat([(sln != nothing) ? abs.(sln.zero)[1:5]' : [NaN, NaN, NaN, NaN, NaN]' for sln in sln_loop_1]...)
gamma_data[:,15:19]  = vcat([(sln != nothing) ? abs.(sln.zero)[1:5]' : [NaN, NaN, NaN, NaN, NaN]' for sln in sln_loop_2]...)

open("gamma_$(γ)_Delta_varies_fixed_P.txt", "w") do io
           writedlm(io, gamma_data)
end